In [4]:
import json
import os
import sys
from typing import Any, TypedDict

# Ajouter le répertoire racine au path pour les imports
sys.path.insert(0, os.path.abspath('..'))



# Configuration LangSmith (optionnelle)
try:
    from backend.config import settings
    from backend.utils.langsmith_init import init_langsmith

    if settings.langsmith_api_key:
        init_langsmith()
        print("✓ LangSmith configuré")
    else:
        print("⚠ LangSmith non configuré (clé API absente)")
except Exception as e:
    print(f"⚠ Configuration LangSmith ignorée: {e}")


# Configuration de l'affichage
def pretty_print(data: Any) -> None:
    """Affiche les données en format JSON lisible."""
    print(json.dumps(data, indent=2, ensure_ascii=False))


print("✓ Configuration initiale terminée")


⚠ LangSmith non configuré (clé API absente)
✓ Configuration initiale terminée


In [ ]:
import re

from langchain_core.messages import AIMessage, BaseMessage, HumanMessage, SystemMessage
from langchain_ollama import ChatOllama
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.graph import END, START, StateGraph


def serialize_messages(messages: list[BaseMessage]) -> list[dict]:
    """Sérialise une liste de messages LangChain en format dictionnaire."""
    return [msg.model_dump() if hasattr(msg, 'model_dump') else msg.dict() for msg in messages]


# Définition de l'état de l'agent
class AgentState(TypedDict):
    """État de l'agent LangGraph pour les transports."""
    adress: str
    messages: list[BaseMessage]
    city_information: dict


memory = InMemorySaver()
llm = ChatOllama(
    model="gemma3",
    temperature=0
    # other params...
)

PROMPT_CITY_INFORMATION = """
Tu reçois en entrée le nom d’une commune française.
À partir de ce nom, produis les trois blocs d’information suivants :

1. Situation :
- Localisation géographique précise (département, région, appartenance
à une agglomération si pertinent).
- Distance approximative aux grandes villes les plus proches.
- Toujours rester factuel.

2. Couleur politique :
- Nom du/de la maire actuel·le.
- Appartenance politique (sigle).
- Année de début du mandat en cours.
- Si l'information est introuvable : indiquer "information non disponible".

3. Présentation qualitative de la commune :
- Description synthétique du caractère de la commune (5–7 lignes).
- Mention d’au moins un élément patrimonial, culturel ou historique notable.
- Présenter clairement les points caractéristiques du territoire.

Format de sortie JSON strict :

{
  "situation": "…",
  "couleur_politique": "…",
  "presentation_qualitative": "…"
}

Toujours répondre en français. Si une donnée est incertaine ou absente, le préciser explicitement.
Entrée : {nom_de_la_commune}
Sortie : l’objet JSON ci-dessus complété.
"""

def extract_json_from_ai_message(ai_msg: AIMessage = None) -> dict:
    content = ai_msg.content

    clean = re.sub(r"^```json|^```|```$", "", content.strip(), flags=re.MULTILINE).strip()

    data = json.loads(clean)

    return data



In [ ]:


def city_information(state: AgentState):
    messages = [
        SystemMessage(PROMPT_CITY_INFORMATION),
        HumanMessage(state["adress"])
    ]

    ai_msg = llm.invoke(messages)

    json_content = extract_json_from_ai_message(ai_msg)
    return {"city_information": json_content}

def create_agent():
    builder = StateGraph(AgentState)
    builder.add_node("city_information", city_information)
    builder.add_edge(START, "city_information")
    builder.add_edge("city_information", END)
    agent = builder.compile()
    return agent




In [ ]:
# Exécution de l'agent
agent = create_agent()
test_input = {
    "adress": "Ballainvilliers",
    "messages": []
}
result = agent.invoke(test_input)



In [ ]:
# result['messages'] = serialize_messages(result["messages"])

from langchain_core.messages import AIMessage

answer: dict = result['city_information']


pretty_print(answer)

# print("Résultat de l'exécution:")
# pretty_print(result)

In [1]:


from backend.agents.graph import market_study_agent

test_input = {
    "adress_in": "Rennes",
    "messages": []
}

result = market_study_agent.invoke(test_input)

In [2]:
print(result)

{'adress_in': 'Rennes', 'messages': [], 'city_information': {'situation': "Rennes est la capitale de la région Bretagne, située dans le département Ille-et-Vilaine. Elle est située sur la rivière Vilaine, à l'extrémité ouest du pays de Rennes. La ville est à environ 30 km de l'aéroport de Nantes Atlantique et à environ 250 km de Paris.  Elle fait partie de l'agglomération rennaise, l'une des plus importantes de France. La ville est un centre économique, administratif et culturel majeur de la région.", 'politique_color': 'information non disponible', 'qualitative_presentation': "Rennes est une ville dynamique et animée, caractérisée par son architecture médiévale et son atmosphère étudiante. Son centre-ville, classé au patrimoine mondial de l'UNESCO, est un ensemble de rues pavées, de places et de monuments historiques. La ville est réputée pour sa vie culturelle riche, avec de nombreux musées, théâtres et salles de concert. Le territoire est composé de quartiers historiques et de zones

In [5]:
answer: dict = result['city_information']


pretty_print(answer)

{
  "situation": "Rennes est la capitale de la région Bretagne, située dans le département Ille-et-Vilaine. Elle est située sur la rivière Vilaine, à l'extrémité ouest du pays de Rennes. La ville est à environ 30 km de l'aéroport de Nantes Atlantique et à environ 250 km de Paris.  Elle fait partie de l'agglomération rennaise, l'une des plus importantes de France. La ville est un centre économique, administratif et culturel majeur de la région.",
  "politique_color": "information non disponible",
  "qualitative_presentation": "Rennes est une ville dynamique et animée, caractérisée par son architecture médiévale et son atmosphère étudiante. Son centre-ville, classé au patrimoine mondial de l'UNESCO, est un ensemble de rues pavées, de places et de monuments historiques. La ville est réputée pour sa vie culturelle riche, avec de nombreux musées, théâtres et salles de concert. Le territoire est composé de quartiers historiques et de zones plus récentes, avec une forte concentration de comme